In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data = data.loc[data['Year']== '2014']
data

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
12957,14756,Wind In Lonely Fences 1970 - 2011,Harold Budd,'s music exists in that misty place between am...,All Saints,7.8,January,8,2014,Experimental,NaN,NaN,NaN
12958,14758,Beyoncé,Beyoncé,"should , choose to settle in the world of adul...",Columbia,8.8,January,6,2014,Pop/R&B,NaN,NaN,NaN
12959,14759,The Abstract And The Dragon,"Busta Rhymes,Q-Tip",the form and function of the hip-hop mixtape h...,Self-released,6.6,January,9,2014,Rap,NaN,NaN,NaN
12960,14760,Spezmodia EP,Mouse on Mars,"when , broke a six-year silence earlier this d...",Monkeytown,7.9,January,9,2014,Electronic,NaN,NaN,NaN
12961,14762,Dirty Gold,Angel Haze,"""sorry to island/republic records,"" 22-year-ol...","Island,Republic",5.9,January,10,2014,Rap,NaN,NaN,NaN
12962,14764,EP-2,Pixies,many groups go away then return to us diminish...,Self-released,2.0,January,10,2014,Rock,NaN,NaN,NaN
12963,14765,Dr. Stokley,Dudley Perkins,you would think that between his long-standing...,Mello,5.8,January,9,2014,Pop/R&B,NaN,NaN,NaN
12964,14767,"The Block Brochure: Welcome To the Soil, Pt. 4...",E-40,"musicians, writers, artists, fellow humans: do...",Heavy on the Grind,7.5,January,6,2014,Rap,NaN,NaN,NaN
12965,14769,Wig Out at Jagbags,Stephen Malkmus and the Jicks,"by now, , has released more music as a solo ar...",Matador,7.0,January,7,2014,Rock,NaN,NaN,NaN
12966,14771,Justus Köhncke & the Wonderful Frequency Band,Justus Köhncke,as cologne’s kompakt ran out the final length ...,Kompakt,6.5,January,7,2014,Electronic,NaN,NaN,NaN


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['content']))

print(data_words[:1])

[['music', 'exists', 'in', 'that', 'misty', 'place', 'between', 'ambient', 'new', 'age', 'and', 'minimalist', 'composition', 'where', 'everything', 'is', 'gentle', 'and', 'nothing', 'lasts', 'for', 'long', 'over', 'the', 'past', 'plus', 'years', 'he', 'released', 'about', 'albums', 'some', 'are', 'solo', 'some', 'are', 'collaborative', 'some', 'are', 'studio', 'and', 'some', 'are', 'live', 'some', 'are', 'improvised', 'and', 'some', 'are', 'structured', 'he', 'famously', 'worked', 'with', 'on', 'the', 'second', 'of', 'eno', 'landmark', 'and', 'less', 'famously', 'made', 'an', 'album', 'with', 'the', 'scottish', 'dream', 'pop', 'band', 'the', 'neither', 'of', 'which', 'have', 'any', 'immediate', 'common', 'ground', 'other', 'than', 'the', 'fact', 'that', 'harold', 'budd', 'plays', 'keyboard', 'on', 'them', 'despite', 'the', 'variety', 'of', 'budd', 'collaborations', 'and', 'circumstances', 'his', 'music', 'has', 'always', 'had', 'quietly', 'recognizable', 'style', 'intimate', 'and', 'in

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['music', 'exists', 'in', 'that', 'misty', 'place', 'between', 'ambient', 'new', 'age', 'and', 'minimalist', 'composition', 'where', 'everything', 'is', 'gentle', 'and', 'nothing', 'lasts', 'for', 'long', 'over', 'the', 'past', 'plus', 'years', 'he', 'released', 'about', 'albums', 'some', 'are', 'solo', 'some', 'are', 'collaborative', 'some', 'are', 'studio', 'and', 'some', 'are', 'live', 'some', 'are', 'improvised', 'and', 'some', 'are', 'structured', 'he', 'famously', 'worked', 'with', 'on', 'the', 'second', 'of', 'eno', 'landmark', 'and', 'less', 'famously', 'made', 'an', 'album', 'with', 'the', 'scottish', 'dream', 'pop', 'band', 'the', 'neither', 'of', 'which', 'have', 'any', 'immediate', 'common', 'ground', 'other', 'than', 'the', 'fact', 'that', 'harold', 'budd', 'plays', 'keyboard', 'on', 'them', 'despite', 'the', 'variety', 'of', 'budd', 'collaborations', 'and', 'circumstances', 'his', 'music', 'has', 'always', 'had', 'quietly', 'recognizable', 'style', 'intimate', 'and', 'int

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['music', 'exist', 'misty', 'place', 'ambient', 'new', 'age', 'minimalist', 'composition', 'everything', 'gentle', 'nothing', 'last', 'long', 'past', 'year', 'release', 'album', 'solo', 'collaborative', 'studio', 'live', 'improvise', 'structure', 'famously', 'work', 'second', 'eno', 'landmark', 'less', 'famously', 'make', 'album', 'scottish', 'dream', 'pop', 'band', 'immediate', 'common', 'ground', 'fact', 'harold', 'play', 'keyboard', 'variety', 'budd', 'collaboration', 'circumstance', 'music', 'always', 'quietly', 'recognizable', 'style', 'intimate', 'intuitive', 'fragile', 'warm', 'seductive', 'little', 'bit', 'mysterious', 'soft', 'tinkle', 'presence', 'next', 'room', 'interview', 'paper', 'joke', 'music', 'tend', 'feel', 'slow', 'searching', 'usually', 'record', 'drunk', 'xcd', 'retrospective', 'compile', 'saint', 'record', 'release', 'vinyl', 'reissue', 'old', 'album', 'album', 'box_set', 'call', 'write', 'convenient', 'inroad', 'budd', 'career', 'ambitious', 'project', 'right',

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 3), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 3), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 3), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 1), (109, 1), (110, 1)

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.053*"blank" + 0.048*"source_material" + 0.032*"firm" + 0.009*"pierce" + '
  '0.001*"score" + 0.001*"film" + 0.000*"movie" + 0.000*"respond" + '
  '0.000*"wail" + 0.000*"loop"'),
 (1,
  '0.110*"romance" + 0.000*"independent" + 0.000*"intensely" + 0.000*"auteur" '
  '+ 0.000*"breathless" + 0.000*"clap" + 0.000*"fancy" + 0.000*"sentimental" + '
  '0.000*"pin" + 0.000*"glow"'),
 (2,
  '0.004*"sort" + 0.004*"noise" + 0.004*"metal" + 0.003*"death" + '
  '0.003*"serve" + 0.003*"ultimately" + 0.003*"room" + 0.003*"mark" + '
  '0.003*"member" + 0.003*"classic"'),
 (3,
  '0.113*"disc" + 0.083*"hi_hat" + 0.060*"uk" + 0.044*"downtempo" + '
  '0.044*"duty" + 0.041*"cheeky" + 0.033*"wife" + 0.027*"throat" + '
  '0.013*"cash" + 0.002*"remaster"'),
 (4,
  '0.090*"print" + 0.000*"distraction" + 0.000*"divide" + 0.000*"stir" + '
  '0.000*"jolt" + 0.000*"advice" + 0.000*"sharpen" + 0.000*"random" + '
  '0.000*"manifesto" + 0.000*"respectively"'),
 (5,
  '0.064*"krautrock" + 0.000*"drill" + 0.00

In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.869201219910995

Coherence Score:  0.3569759879491181


In [13]:
# Visualize the topics
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#vis

In [14]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print(topic_idx)
        print.join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print (documents[doc_index])